In [1]:
import os

import torch

In [5]:
from PyPDF2 import PdfReader

In [6]:
reader = PdfReader("/mnt/c/Users/mikol/Desktop/kiid_makro_alokacji_kat._b.pdf")

In [7]:
page = reader.pages[0]

In [8]:
page.extract_text()

'Kluczowe informacje dla inwestorów  \nNiniejszy dokument zawiera kluczowe informacje dla inwestorów dotyczące tego funduszu. Nie są to materiały marketingowe. \nDostarczenie  tych informacji jest wymogiem prawnym mającym na celu ułatwienie zrozumienia charakteru i ryzyka związanego \nz inwestowaniem w ten fundusz. Przeczytanie niniejszego dokumentu jest zalecane inwestorowi, aby mógł on podjąć świadomą decyzj ę \ninwestycyjną.  \n \nBaltic Makro Alokacji  („Subfundusz”) , jedn ostki uczestnictwa kategori i B \nSubfundusz wydzielony w ramach Baltic Specjalistyczny  Fundusz Inwestycyjny Otwarty  \nFundusz jest zarządzany przez spółkę Baltic Capital  Towarzystwo Funduszy Inwestycyjnych S.A.  \n \nCele i polityka inwestycyjna  \nCelem inwestycyjnym Su bfunduszu jest wzrost wartości \nAktywów Subfunduszu w wyniku wzrostu wartości lokat.  \nZ uwzględnieniem zda ń następn ych, Fundusz lokuje do 60% \nwartości Aktywów Subfunduszu  w instrumenty udziałowe, w tym \nw szczególności akcje dopuszc

In [10]:
with open('linki','r') as file:
    linki = file.read().split('\n')

In [14]:
import requests
resp = requests.get('https://www.esaliens.pl/~/media/files/pobierz-dokumenty/fs/esa-akcji/2022-09-kah02-esaliens-akcji.ashx')


In [16]:
resp = requests.get('https://agiofunds.pl')

In [19]:
from bs4 import BeautifulSoup as bs
soup = bs(resp.content)
soup.find_all('a', recursive=True, href=True)

[<a class="lang_sel_sel" href="https://agiofunds.pl"><img alt="pl" class="iclflag" src="https://agiofunds.pl/wp-content/plugins/sitepress-multilingual-cms/res/flags/pl.png" title="polski"/> </a>,
 <a class="lang_sel_other" href="https://agiofunds.pl/?lang=en"><img alt="en" class="iclflag" src="https://agiofunds.pl/wp-content/plugins/sitepress-multilingual-cms/res/flags/en.png" title="English"/> </a>,
 <a class="phone" href="tel:48225315454" title="T: +48 22 531 54 54">T: +48 22 531 54 54</a>,
 <a class="toplogo-link" href="https://agiofunds.pl" title="AgioFunds"><img alt="AgioFunds" class="toplogo" src="https://agiofunds.pl/wp-content/uploads/2016/06/logo-1-e1575040107915.png"/></a>,
 <a href="https://agiofunds.pl/inwestycje/">Inwestycje</a>,
 <a href="https://agiofunds.pl/fundusze-inwestycyjne/">Fundusze inwestycyjne</a>,
 <a href="https://agiofunds.pl/fundusze-dedykowane/">Fundusze dedykowane</a>,
 <a href="https://agiofunds.pl/strategie-inwestycyjne/">Strategie inwestycyjne</a>,
 <a

In [20]:
resp.headers.get('Content-Type')

'text/html; charset=UTF-8'

In [23]:
import re
re.findall('dupa', 'smierdzDuPaimidudpakupa', flags=re.I)

['DuPa']

In [2]:
import requests
from bs4 import BeautifulSoup as bs
resp = requests.get('https://esaliens.pl')
soup = bs(resp.content, 'html.parser')
aki = soup.find_all('a', href=True, recursive=True)

In [3]:
aki[0].get_text()

' Treść polityki dotyczącej korzystania z plików cookies'

In [10]:

import requests
import re
from bs4 import BeautifulSoup as bs
import logging
import sys
from urllib.parse import urlparse, urlunparse
from PyPDF2 import PdfReader as pdfr
import io

from requests.exceptions import MissingSchema, InvalidSchema

logging.basicConfig(
    stream=sys.stdout,
    level=logging.DEBUG,
    format="%(asctime)s-%(levelname)s:%(message)s"
)

logger = logging.getLogger(__name__)


def is_not_bull(a):
    return a.get('href') not in ('/', '#') and not re.match(r'^tel:', a.get('href')) and not re.match(r'^mail:', a.get('href')) and not re.match(r'^mailto:', a.get('href'))

def is_onsite(a, site):
    if not urlparse(a.get('href')).netloc or urlparse(a.get('href')).netloc == urlparse(site):
        return True
    return False

def add_netloc(a, site):
    link = a.get('href')
    parsed_link = urlparse(link)
    parsed_site = urlparse(site)
    if not parsed_link.netloc:
        a['href'] = urlunparse(
            (
                parsed_site.scheme,
                parsed_site.netloc,
                parsed_link.path,
                parsed_link.params,
                '',
                ''
            )
        )
    return a

def is_not_seen(a, seen):
    if a.get('href') in seen:
        return False
    return True

def is_check(a, site, seen):
    if is_not_bull(a) and is_onsite(a, site) and is_not_seen(a, seen):
        return True
    return False

def is_to_save(a):
    text = a.get_text()
    if re.match(r'.*KII.*', text, flags=re.I) or re.match(r'.*karta.*', text, flags=re.I) or re.match(r'.*kluczowe.*inwestorów.*', text, flags=re.I):
        return True
    return False

def dfs(link):
    """

    :param link: valid url
    :return: list of <a> tags from all pages found on site
    """
    seen = set()
    def dfs_(a):
        if not isinstance(a, str):
            link_ = a.get('href')
        else:
            link_ = a
        seen.add(link_)
        logger.info(f"trying {link_}")
        try:
            resp = requests.get(link_)
            if resp.status_code != 200: # noqa
                logger.warning(f"GET {link_} failed with code: {resp.status_code}")
                return []
            logger.debug(f"GET {link_} content-type: {resp.headers.get('Content-Type')}")
            if resp.headers.get("Content-Type", "").__contains__("text/html"):
                soup = bs(resp.content)
                found_as = soup.find_all("a", href=True, recursive=True)
                to_check_as = [add_netloc(a, link) for a in found_as if is_check(a, link, seen)]
                for a in to_check_as:
                    if is_not_seen(a, seen):
                        dfs_(a)

            elif resp.headers.get("Content-Type", "").__contains__("application/pdf"):
                if is_to_save(a):
                    if len(pdfr(io.BytesIO(resp.content)).pages) == 2:
                        with open(f'../data/raw/{a.get("href").split("/")[-1]}', 'wb') as file:
                            for chunk in resp.iter_content(chunk_size=1024):
                                file.write(chunk)

            else:
                logger.warn(f"unhandled content-type: {resp.headers.get('Content-Type')}")
        except (MissingSchema, InvalidSchema) as exc:
            logger.warning(f"GET {link_} failed with {exc}")


    dfs_(link)


In [ ]:
with open('linki', 'r') as file:
    linki = file.read().split('\n')
for link in linki:
    dfs(link)


2022-10-22 13:55:23,885-INFO:trying http://www.agiofunds.pl
2022-10-22 13:55:23,887-DEBUG:Starting new HTTP connection (1): www.agiofunds.pl:80
2022-10-22 13:55:23,902-DEBUG:http://www.agiofunds.pl:80 "GET / HTTP/1.1" 301 233
2022-10-22 13:55:23,904-DEBUG:Starting new HTTPS connection (1): www.agiofunds.pl:443
2022-10-22 13:55:24,898-DEBUG:https://www.agiofunds.pl:443 "GET / HTTP/1.1" 301 0
2022-10-22 13:55:24,901-DEBUG:Starting new HTTPS connection (1): agiofunds.pl:443
2022-10-22 13:55:27,106-DEBUG:https://agiofunds.pl:443 "GET / HTTP/1.1" 200 None
2022-10-22 13:55:27,145-DEBUG:GET http://www.agiofunds.pl content-type: text/html; charset=UTF-8
2022-10-22 13:55:27,195-INFO:trying http://www.agiofunds.pl/dokument
2022-10-22 13:55:27,197-DEBUG:Starting new HTTP connection (1): www.agiofunds.pl:80
2022-10-22 13:55:27,214-DEBUG:http://www.agiofunds.pl:80 "GET /dokument HTTP/1.1" 301 241
2022-10-22 13:55:27,215-DEBUG:Starting new HTTPS connection (1): www.agiofunds.pl:443
2022-10-22 13:55: